In [1]:
from opensky_api import OpenSkyApi, FlightTrack
import os.path
import pandas as pd
import folium
import time
import os 
from dotenv import load_dotenv
from folium.plugins import MarkerCluster
load_dotenv()
import avwx
opensky_user = os.getenv('OPENSKY_USER')
opensky_api_key = os.getenv('OPENSKY_API_KEY')

# Initialize the API (can also be authenticated) 
# dont be dumb if you have the keys
api = OpenSkyApi(opensky_user, opensky_api_key)

# Fetch Live Flights in the Air or on the Ground in the United States

In [2]:
# fetch flights only in the united states 
states = api.get_states()
data = []
for s in states.states:
    data.append({
            "ICAO24": s.icao24,
            "Callsign": s.callsign,
            "Origin Country": s.origin_country,
            "Latitude": s.latitude,
            "Longitude": s.longitude,
            "Baro Altitude": s.baro_altitude,
            "Geo Altitude": s.geo_altitude,
            "Velocity": s.velocity,
            "True Track": s.true_track,
            "Vertical Rate": s.vertical_rate,
            "On Ground": s.on_ground,
            "Category": s.category,
            "Last Contact": s.last_contact,
            "Position Source": s.position_source,
            "Sensors": s.sensors,
            "SPI": s.spi,
            "Squawk": s.squawk,
            "Time Position": s.time_position
        })

us_flights_data = [flight for flight in data if flight["Origin Country"] == "United States"]
us_flights_df = pd.DataFrame(us_flights_data)
us_flights_df

,ICAO24,Callsign,Origin Country,Latitude,Longitude,Baro Altitude,Geo Altitude,Velocity,True Track,Vertical Rate,On Ground,Category,Last Contact,Position Source,Sensors,SPI,Squawk,Time Position
0,ab1644,UAL1777,United States,40.8735,-75.0839,2651.76,2667.00,167.33,110.33,-7.15,False,0,1713215533,0,None,False,1525,1.713216e+09
1,aa8c39,UAL1758,United States,36.8572,-125.4615,10363.20,10607.04,248.42,251.90,0.00,False,0,1713215260,0,None,False,3261,1.713215e+09
2,ac96b8,AAL1180,United States,31.7294,-95.9298,9029.70,9479.28,226.14,328.30,-5.20,False,0,1713215533,0,None,False,None,1.713216e+09
3,a1abf5,N2069W,United States,14.5952,-90.5905,1866.90,1950.72,76.18,271.93,-0.33,False,0,1713215532,0,None,False,2736,1.713216e+09
4,a46e70,LXJ385,United States,49.1856,-123.1681,-91.44,38.10,5.40,174.38,NaN,False,0,1713215529,0,None,False,3275,1.713215e+09
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6043,a4e5f6,ASI553,United States,33.9043,-112.4453,1645.92,1645.92,51.70,95.71,-4.23,False,0,1713215532,0,None,False,None,1.713216e+09
6044,a12403,UAL2196,United States,41.4406,-80.2738,11277.60,11384.28,252.99,74.55,0.00,False,0,1713215533,0,None,False,7264,1.713216e+09
6045,abc0e4,SWA392,United States,40.2835,-80.6905,10584.18,10736.58,200.68,271.18,7.80,False,0,1713215533,0,None,False,6216,1.713216e+09
6046,a4e5ea,ILU44K,United States,40.0302,-88.2871,251.46,243.84,36.79,42.17,-1.63,False,0,1713215394,0,None,False,None,1.713215e+09


# Load US airport data

In [3]:
# Load the airport data from the CSV file
airport_df = pd.read_csv('airports.csv')
# US only airports
us_airports_df = airport_df[airport_df['iso_country'] == 'US']
# now only get medium airport and large airports
us_airports_df = us_airports_df[us_airports_df['type'].isin(['medium_airport', 'large_airport'])]
us_airports_df

,ident,type,name,elevation_ft,continent,iso_country,iso_region,municipality,gps_code,iata_code,local_code,coordinates
6220,5A8,medium_airport,Aleknagik / New Airport,66.0,NaN,US,US-AK,Aleknagik,5A8,WKK,5A8,"-158.617996216, 59.2826004028"
27220,K79J,medium_airport,South Alabama Regional At Bill Benton Field Ai...,310.0,NaN,US,US-AL,Andalusia/Opp,K79J,NaN,79J,"-86.393799, 31.3088"
27399,KABE,medium_airport,Lehigh Valley International Airport,393.0,NaN,US,US-PA,Allentown,KABE,ABE,ABE,"-75.44080352783203, 40.652099609375"
27400,KABI,medium_airport,Abilene Regional Airport,1791.0,NaN,US,US-TX,Abilene,KABI,ABI,ABI,"-99.68190002440001, 32.4113006592"
27401,KABQ,large_airport,Albuquerque International Sunport,5355.0,NaN,US,US-NM,Albuquerque,KABQ,ABQ,ABQ,"-106.609001, 35.040199"
...,...,...,...,...,...,...,...,...,...,...,...,...
40771,PPIZ,medium_airport,Point Lay LRRS Airport,22.0,NaN,US,US-AK,Point Lay,PPIZ,PIZ,PIZ,"-163.0050049, 69.73290253"
50994,US-0086,medium_airport,Jim's Private Airport,890.0,NaN,US,US-GA,Conyers,NaN,NaN,NaN,"-84.14440155, 33.59790039"
51162,US-0254,medium_airport,Yuma Auxiliary Army Airfield 2,NaN,NaN,US,US-AZ,Yuma,NaN,NaN,NaN,"-114.511786, 32.547501"
51479,US-0571,medium_airport,Williston Basin International Airport,2344.0,NaN,US,US-ND,Williston,KXWA,XWA,XWA,"-103.748797, 48.258387"


In [4]:
# m = folium.Map(location=[37, -102])
# marker_cluster = MarkerCluster().add_to(m)

# for i, row in us_airports_df.iterrows():
#     folium.Marker([row['Latitude'], row['Longitude']], popup=row['Callsign']).add_to(marker_cluster)

# m.save('us_airports.html')

# Weather: AVWX library

In [5]:
import avwx
from avwx.forecast import *
import asyncio

# Function to Fetch Data
async def fetch_airport_data_async(airport_codes):
    airport_data = {}
    for airport_code in airport_codes:
        data = {}
        # Fetch METAR data
        try:
            metar = avwx.Metar(airport_code)
            if await metar.async_update():
                data["metar"] = {
                    "raw": metar.translations,
                    "flight_rules": metar.data.flight_rules,
                    "summary": metar.summary, 
                }
            else:
                data["metar"] = None
        except Exception as e:
            print(f"Error fetching METAR data for {airport_code}: {e}")
            data["metar"] = None

        # Fetch TAF data
        try:
            taf = avwx.Taf(airport_code)
            if await taf.async_update():
                data["taf"] = {
                    "raw": taf.translations,
                    "forecast": [
                        {
                            "flight_rules": line.flight_rules,
                            "start_time": line.start_time.dt.strftime('%d-%H:%M'),
                            "end_time": line.end_time.dt.strftime('%d-%H:%M'),
                        }
                        for line in taf.data.forecast
                    ],
                }
            else:
                data["taf"] = None
        except Exception as e:
            print(f"Error fetching TAF data for {airport_code}: {e}")
            data["taf"] = None
        
        airport_data[airport_code] = data
    return airport_data


async def main():
    airports = us_airports_df['gps_code'].tolist()
    data = await fetch_airport_data_async(['KHHR', 'KSBA', 'KSQL', 'KSJC', 'KTRK'])
    # html_output = generate_summary_html_safe(data)
#     # with open("weather_summary.html", "w") as file:
#     #     file.write(html_output)
    return data  # Return the data instead of printing it

# Run the Main Function
if __name__ == "__main__":
    import nest_asyncio
    nest_asyncio.apply()
    loop = asyncio.get_event_loop()
    data = loop.run_until_complete(main()) 

## Mapping weather by airport

In [6]:
import folium
import pandas as pd

def display_airport_weather_map_with_taf(us_airports_df, weather_data):
    # Initialize a folium map centered on the United States
    m = folium.Map(location=[37.0902, -95.7129], zoom_start=4)
    
    for airport_code, info in weather_data.items():
        if airport_code in us_airports_df['gps_code'].values:
            airport_row = us_airports_df[us_airports_df['gps_code'] == airport_code]
            coordinates = airport_row['coordinates'].values[0].split(', ')
            latitude = float(coordinates[1])
            longitude = float(coordinates[0])
            
            metar_translated = info['metar']['summary'] if 'metar' in info and 'summary' in info['metar'] else 'No METAR summary available'
            flight_rules = info['metar']['flight_rules'] if 'metar' in info and 'flight_rules' in info['metar'] else 'Unknown'

            # Start HTML for popup
            popup_html = f"<b>{airport_code}</b><br>{metar_translated}<br>Flight Rules: {flight_rules}<br><br><b>TAF Forecast:</b>"

            # Check if TAF data is available
            if 'taf' in info and info['taf']:
                taf_details = info['taf']
                popup_html += f"<p>TAF Details: {taf_details}</p>"
            else:
                popup_html += "<p>No TAF details available.</p>"
            # Check if TAF data is available
            if 'taf' in info and info['taf'] and 'forecast' in info['taf']:
                popup_html += "<table><tr><th>Time</th><th>Flight Rules</th></tr>"
                for forecast in info['taf']['forecast']:
                    start_time = forecast.get('start_time', 'N/A')
                    end_time = forecast.get('end_time', 'N/A')
                    fr = forecast.get('flight_rules', 'Unknown')
                    popup_html += f"<tr><td>{start_time} to {end_time}</td><td>{fr}</td></tr>"
                popup_html += "</table>"
            else:
                popup_html += "<p>No TAF forecast available.</p>"

            color_map = {
                'VFR': 'green',
                'MVFR': 'blue',
                
                'IFR': 'orange',
                'LIFR': 'red'
            }
            color = color_map.get(flight_rules, 'gray')
            
            iframe = folium.IFrame(html=popup_html, width=300, height=200)
            popup = folium.Popup(iframe, max_width=300)

            folium.Marker(
                location=[latitude, longitude],
                popup=popup,
                icon=folium.Icon(color=color)
            ).add_to(m)
    
    return m

# Assuming 'us_airports_df' and 'data' are defined
m = display_airport_weather_map_with_taf(us_airports_df, data)
m


## Find Icing Conditions

In [7]:
# Initialize a list to hold airports with icing conditions
airports_with_icing = []

# Iterate through the dictionary
for airport_code, weather_info in data.items():
    if 'taf' in weather_info and weather_info['taf'] is not None:  # Ensure TAF data is available
        for forecast_line in weather_info['taf']['raw'].forecast:  # Loop through each forecast line
            if forecast_line.icing:  # Check if icing is mentioned
                # Since icing is an empty string in your example, you'd check its presence differently
                # This is a placeholder condition, adjust according to how icing data is represented
                if forecast_line.icing != '':  # Adjust this condition based on actual data representation
                    airports_with_icing.append(airport_code)
                    break  # Stop checking further lines for this airport

# Now, 'airports_with_icing' contains the codes of airports with icing conditions
print("Airports with icing conditions:", airports_with_icing)

Airports with icing conditions: []


## Generates HTML for airport TAFs



In [ ]:
# # Function to Generate HTML
# def generate_summary_html_safe(airports_data):
#     # Styles
#     styles = """
#     <style>
#         table {
#             border-collapse: collapse;
#             width: 80%;
#             margin: 20px auto;
#         }
#         th, td {
#             text-align: left;
#             padding: 8px 12px;
#             border: 1px solid #ddd;
#         }
#         tr:nth-child(even) {
#             background-color: #f4f4f4;
#         }
#         th {
#             background-color: #f2f2f2;
#         }
#         .airport-name {
#             font-weight: bold;
#             font-size: 1.5em;
#             text-align: center;
#             margin-top: 20px;
#         }
#         .section-header {
#             font-weight: bold;
#             margin-top: 40px;
#             font-size: 1.3em;
#             text-align: center;
#         }
#     </style>
#     """
    
#     # Rows
#     metar_rows_html = ""
#     taf_rows_html = ""
#     for airport, data in airports_data.items():
#         # METAR
#         if data["metar"]:
#             metar = data["metar"]["summary"].split(", ")
#             metar_rows_html += f"<tr>"
#             metar_rows_html += f"<td>{airport}</td>"
#             metar_rows_html += f"<td>{data['metar']['flight_rules']}</td>"
#             for item in metar:
#                 metar_rows_html += f"<td>{item}</td>"
#             metar_rows_html += "</tr>"
        
#         # TAF
#         if data["taf"]:
#             taf_rows_html += f"<tr>"
#             taf_rows_html += f"<td>{airport}</td>"
#             taf_forecast = ', '.join([f"{forecast['flight_rules']} from {forecast['start_time']} to {forecast['end_time']}" for forecast in data['taf']['forecast']])
#             taf_rows_html += f"<td>{taf_forecast}</td>"
#             taf_rows_html += "</tr>"

#     # Complete HTML Structure
#     html = f"""
#     <html>
#         <head>
#             <title>Weather Summary</title>
#             {styles}
#         </head>
#         <body>
#             <div class='airport-name'>Weather Summary for {', '.join(airports_data.keys())}</div>
#             <div class='section-header'>METAR</div>
#             <table>
#                 <tr><th>Airport</th><th>Flight Rules</th><th>Wind</th><th>Visibility</th><th>Temperature</th><th>Dew Point</th><th>Pressure</th><th>Clouds</th><th>Conditions</th></tr>
#                 {metar_rows_html}
#             </table>
#             <div class='section-header'>TAF</div>
#             <table>
#                 <tr><th>Airport</th><th>Forecast</th></tr>
#                 {taf_rows_html}
#             </table>
#         </body>
#     </html>
#     """
#     return html

# Async Main Function to Drive the Process
# Async Main Function to Drive the Process